In [0]:
%sql
-- C-NOS
 
WITH base AS (
    SELECT
        category,
        forecast_act_est,           -- decimal(22,8)
        ya_actual,                  -- decimal(22,8)
        target,                     -- decimal(22,8)
        period,
        month
    FROM japan_gold.cnos_vw
    WHERE category IS NOT NULL
      AND customer IS NOT NULL
      AND account = 'GIV'
),
 
-- Pick the most recent period present in the view
current_period AS (
    SELECT
        MAX(CASE WHEN month = CONCAT(date_format(current_date(), 'yyyy'), 'M', date_format(current_date(), 'MM')) THEN period END) AS period
    FROM base
),
 
-- Final aggregation by category (if multiple rows per category exist for the selected period, we sum them)
curr_agg AS (
    SELECT
        category,
        SUM(COALESCE(forecast_act_est, 0)) AS forecast_act_est,
        SUM(COALESCE(ya_actual, 0))        AS ya_actual,
        SUM(COALESCE(target, 0))           AS target
    FROM base b
    JOIN current_period cp
      ON b.period = cp.period
    GROUP BY category
)
 
 
-- ==== BEGIN: your existing query exactly as-is ====
SELECT *
FROM (
  (SELECT
      'TOTAL' AS `Category`,
      ROUND(SUM(c.target) / NULLIF(SUM(c.ya_actual), 0) * 100, 0) AS `Target IYA`,
      ROUND(SUM(c.forecast_act_est) / 1000000.0 / 150.1, 1)       AS `1H MM`,
      ROUND(SUM(c.forecast_act_est) / NULLIF(SUM(c.ya_actual), 0) * 100, 0) AS `1H IYA`,
      ROUND((SUM(c.forecast_act_est) - SUM(c.target)) / 1000000.0 / 150.1, 1) AS `1H Gap to Target MM`,
      ROUND(
          (SUM(c.forecast_act_est) / NULLIF(SUM(c.ya_actual), 0) * 100)
          - (SUM(c.target) / NULLIF(SUM(c.ya_actual), 0) * 100), 0
      ) AS `1H Gap to Target IYA`
   FROM curr_agg c
 )
  UNION
  (SELECT
      c.category AS `Category`,
      ROUND(c.target / NULLIF(c.ya_actual, 0) * 100, 0) AS `Target IYA`,
      ROUND(c.forecast_act_est / 1000000.0 / 150.1, 1)  AS `1H MM`,
      ROUND(c.forecast_act_est / NULLIF(c.ya_actual, 0) * 100, 0) AS `1H IYA`,
      ROUND((c.forecast_act_est - c.target) / 1000000.0 / 150.1, 1) AS `1H Gap to Target MM`,
      ROUND(
          (c.forecast_act_est / NULLIF(c.ya_actual, 0) * 100)
          - (c.target / NULLIF(c.ya_actual, 0) * 100), 0
      ) AS `1H Gap to Target IYA`
   FROM curr_agg c
  )
) t
WHERE t.`Category` IN (
  'TOTAL',
  'Detergent',
  'Fabric Enhancer',
  'Dish',
  'Air',
  'Baby',
  'Fem',
  'Hair',
  'Shave',
  'Oral',
  'Braun',
  'POC'
)
ORDER BY
  CASE t.`Category`
    WHEN 'TOTAL'            THEN 0
    WHEN 'Detergent'        THEN 1
    WHEN 'Fabric Enhancer'  THEN 2
    WHEN 'Dish'             THEN 3
    WHEN 'Air'              THEN 4
    WHEN 'Baby'             THEN 5
    WHEN 'Fem'              THEN 6
    WHEN 'Hair'             THEN 7
    WHEN 'Shave'            THEN 8
    WHEN 'Oral'             THEN 9
    WHEN 'Braun'            THEN 10
    WHEN 'POC'              THEN 11
    ELSE 99
  END;

In [0]:
%sql

-- POS

WITH params AS (
    SELECT
        -- First day of the current month (anchor)
        to_date(date_trunc('month', current_date())) AS anchor_month_start,
        -- Last day of the last fully completed month
        date_sub(to_date(date_trunc('month', current_date())), 1) AS last_full_month_end
),
ranges AS (
    SELECT
        -- Current windows (end at last_full_month_end)
        add_months(anchor_month_start, -1) AS p1m_start,
        add_months(anchor_month_start, -3) AS p3m_start,
        add_months(anchor_month_start, -6) AS p6m_start,
        last_full_month_end              AS curr_end,

        -- YA windows (same spans shifted by -12 months)
        add_months(add_months(anchor_month_start, -1), -12) AS p1m_ya_start,
        add_months(add_months(anchor_month_start, -3), -12) AS p3m_ya_start,
        add_months(add_months(anchor_month_start, -6), -12) AS p6m_ya_start,
        add_months(last_full_month_end, -12)                AS ya_end
    FROM params
),
fact AS (
    SELECT
        pd.jp_category_name                        AS category,
        CAST(f.day_date AS DATE)                   AS day_date,
        COALESCE(f.pos_sales_amt, 0)               AS pos_sales_amt,
        case when pd.jp_mfgr_name='P&G' then f.pos_sales_amt end as PG_pos_sales_amt
    FROM japan_gold.pos_monthly_fct_vw f
     JOIN japan_gold.product_dim_vw pd
        ON pd.prod_key = f.prod_key
        -- Join site_dim_vw via site_key
 JOIN japan_gold.site_dim_vw s
ON s.site_key = f.site_key
-- Join customer_japan_dim_vw via site_dim_vw.own_party_key -> customer.cust_key
 JOIN japan_gold.customer_japan_dim_vw c
ON c.cust_key = s.own_party_key
-- Apply required filter: jp_cust_lvl_1_name is not blank
WHERE c.jp_cust_lvl_1_name IS NOT NULL
	and jp_category_name not in ('Super Premium Skin Care','P&G non playing')
    -- Optional scoping:
    -- WHERE f.secure_group_key = 'your_group'
    --   AND f.data_provider_key IN (...)
    --   AND f.site_key IN (...)
),
agg AS (
    SELECT
        f.category,

        -- Current windows: sums ending at last_full_month_end
        SUM(CASE WHEN f.day_date BETWEEN r.p1m_start AND r.curr_end THEN f.pos_sales_amt ELSE 0 END) AS p1m_curr,
        SUM(CASE WHEN f.day_date BETWEEN r.p3m_start AND r.curr_end THEN f.pos_sales_amt ELSE 0 END) AS p3m_curr,
        SUM(CASE WHEN f.day_date BETWEEN r.p6m_start AND r.curr_end THEN f.pos_sales_amt ELSE 0 END) AS p6m_curr,
        SUM(CASE WHEN f.day_date BETWEEN r.p1m_start AND r.curr_end THEN f.PG_pos_sales_amt ELSE 0 END) AS PG_p1m_curr,
        SUM(CASE WHEN f.day_date BETWEEN r.p3m_start AND r.curr_end THEN f.PG_pos_sales_amt ELSE 0 END) AS PG_p3m_curr,
        SUM(CASE WHEN f.day_date BETWEEN r.p6m_start AND r.curr_end THEN f.PG_pos_sales_amt ELSE 0 END) AS PG_p6m_curr,

        -- YA windows: same ranges shifted -12 months
        SUM(CASE WHEN f.day_date BETWEEN r.p1m_ya_start AND r.ya_end THEN f.pos_sales_amt ELSE 0 END) AS p1m_ya,
        SUM(CASE WHEN f.day_date BETWEEN r.p3m_ya_start AND r.ya_end THEN f.pos_sales_amt ELSE 0 END) AS p3m_ya,
        SUM(CASE WHEN f.day_date BETWEEN r.p6m_ya_start AND r.ya_end THEN f.pos_sales_amt ELSE 0 END) AS p6m_ya,
        SUM(CASE WHEN f.day_date BETWEEN r.p1m_ya_start AND r.ya_end THEN f.PG_pos_sales_amt ELSE 0 END) AS PG_p1m_ya,
        SUM(CASE WHEN f.day_date BETWEEN r.p3m_ya_start AND r.ya_end THEN f.PG_pos_sales_amt ELSE 0 END) AS PG_p3m_ya,
        SUM(CASE WHEN f.day_date BETWEEN r.p6m_ya_start AND r.ya_end THEN f.PG_pos_sales_amt ELSE 0 END) AS PG_p6m_ya
    FROM fact f
    CROSS JOIN ranges r
    GROUP BY f.category
)
SELECT
    'TOTAL' AS `Category`,

    ROUND(sum(p1m_curr) / NULLIF(sum(p1m_ya), 0)*100, 1) AS `P1M`,
    ROUND(sum(p3m_curr) / NULLIF(sum(p3m_ya), 0)*100, 1) AS `P3M`,
    ROUND(sum(p6m_curr )/ NULLIF(sum(p6m_ya), 0)*100, 1) AS `P6M`,
    ROUND(sum(PG_p1m_curr) / NULLIF(sum(PG_p1m_ya), 0)*100, 1) AS `PG_P1M`,
    ROUND(sum(PG_p3m_curr )/ NULLIF(sum(PG_p3m_ya), 0)*100, 1) AS `PG_P3M`,
    ROUND(sum(PG_p6m_curr) / NULLIF(sum(PG_p6m_ya), 0)*100, 1) AS `PG_P6M`
FROM agg
UNION

SELECT
    category AS `Category`,
    -- "Category POS Value IYA" merged header over these sub-columns in your BI tool
    ROUND(p1m_curr / NULLIF(p1m_ya, 0)*100, 1) AS `P1M`,
    ROUND(p3m_curr / NULLIF(p3m_ya, 0)*100, 1) AS `P3M`,
    ROUND(p6m_curr / NULLIF(p6m_ya, 0)*100, 1) AS `P6M`,
    ROUND(PG_p1m_curr / NULLIF(PG_p1m_ya, 0)*100, 1) AS `PG_P1M`,
    ROUND(PG_p3m_curr / NULLIF(PG_p3m_ya, 0)*100, 1) AS `PG_P3M`,
    ROUND(PG_p6m_curr / NULLIF(PG_p6m_ya, 0)*100, 1) AS `PG_P6M`
FROM agg;

In [0]:
%sql

-- POS Share

--US32603 insightsPG report for Sufficiency Review (POS)
-- Category POS Share + DYA (P1M, P3M, P6M) + vs National Share (P1M delta) using calendar months

WITH params AS (
    SELECT
        to_date(date_trunc('month', current_date())) AS anchor_month_start,
        date_sub(to_date(date_trunc('month', current_date())), 1) AS last_full_month_end
),
ranges AS (
    SELECT
        add_months(anchor_month_start, -1) AS p1m_start,
        add_months(anchor_month_start, -3) AS p3m_start,
        add_months(anchor_month_start, -6) AS p6m_start,
        last_full_month_end              AS curr_end,

        add_months(add_months(anchor_month_start, -1), -12) AS p1m_ya_start,
        add_months(add_months(anchor_month_start, -3), -12) AS p3m_ya_start,
        add_months(add_months(anchor_month_start, -6), -12) AS p6m_ya_start,
        add_months(last_full_month_end, -12)                AS ya_end
    FROM params
),

/* -------------------- CATEGORY POS SOURCE (your POS facts) -------------------- */
fact AS (
    SELECT
        pd.jp_category_name              AS category,
        pd.jp_sub_category_name          AS sub_category,
        pd.jp_local_pos_share_flag       AS in_flag,
        pd.jp_mfgr_name                  AS mfgr,
        CAST(f.day_date AS DATE)         AS day_date,
        COALESCE(f.pos_sales_amt, 0)     AS pos_sales_amt
    FROM japan_gold.pos_monthly_fct_vw f
    JOIN japan_gold.product_dim_vw pd
      ON pd.prod_key = f.prod_key
    -- Uncomment if you need customer/site scoping
    -- JOIN japan_gold.site_dim_vw s ON s.site_key = f.site_key
    -- JOIN japan_gold.customer_japan_dim_vw c ON c.cust_key = s.own_party_key
    -- WHERE c.jp_cust_lvl_1_name IS NOT NULL
),

/* -------------------- NATIONAL MARKET SOURCE (intage) -------------------- */
fact_intage AS (
    SELECT 
        pd.jp_category_name              AS category,              -- not used for national total
        pd.jp_local_pos_share_flag       AS in_flag,
        pd.jp_mfgr_name                  AS mfgr,
        CAST(intage.day_date AS DATE)    AS day_date,
        COALESCE(intage.sales_mlc_amt, 0) AS sales_mlc_amt
    FROM japan_gold.intage_mth_fct_vw intage
    JOIN japan_gold.intage_prod_dim_vw ipd
      ON ipd.prod_skid = intage.prod_skid
    JOIN japan_gold.product_dim_vw pd
      ON pd.prod_key = ipd.prod_key
),

/* -------------------- NATIONAL P1M SHARE (scalar) -------------------- */
nat AS (
    SELECT
    f.category,
        /* Numerator: 'in' AND mfgr cohort */
        SUM(CASE WHEN f.day_date BETWEEN r.p1m_start AND r.curr_end
                  AND f.in_flag = 'in'
                  AND f.mfgr IN ('P&G','KAO','LION')
                 THEN f.sales_mlc_amt ELSE 0 END) AS num_nat_p1m_curr,
        /* Denominator: 'in' only */
        SUM(CASE WHEN f.day_date BETWEEN r.p1m_start AND r.curr_end
                  AND f.in_flag = 'in'
                 THEN f.sales_mlc_amt ELSE 0 END) AS den_nat_p1m_curr
    FROM fact_intage f
    CROSS JOIN ranges r
    GROUP by f.category
),

/* -------------------- CATEGORY POS SHARE COMPONENTS -------------------- */
agg AS (
    SELECT
        f.category,

        /* Current windows */
        SUM(CASE WHEN f.day_date BETWEEN r.p1m_start AND r.curr_end
                  AND f.in_flag = 'in' AND f.mfgr IN ('P&G','KAO','LION')
                 THEN f.pos_sales_amt ELSE 0 END) AS num_p1m_curr,
        SUM(CASE WHEN f.day_date BETWEEN r.p3m_start AND r.curr_end
                  AND f.in_flag = 'in' AND f.mfgr IN ('P&G','KAO','LION')
                 THEN f.pos_sales_amt ELSE 0 END) AS num_p3m_curr,
        SUM(CASE WHEN f.day_date BETWEEN r.p6m_start AND r.curr_end
                  AND f.in_flag = 'in' AND f.mfgr IN ('P&G','KAO','LION')
                 THEN f.pos_sales_amt ELSE 0 END) AS num_p6m_curr,

        SUM(CASE WHEN f.day_date BETWEEN r.p1m_start AND r.curr_end
                  AND f.in_flag = 'in'
                 THEN f.pos_sales_amt ELSE 0 END) AS den_p1m_curr,
        SUM(CASE WHEN f.day_date BETWEEN r.p3m_start AND r.curr_end
                  AND f.in_flag = 'in'
                 THEN f.pos_sales_amt ELSE 0 END) AS den_p3m_curr,
        SUM(CASE WHEN f.day_date BETWEEN r.p6m_start AND r.curr_end
                  AND f.in_flag = 'in'
                 THEN f.pos_sales_amt ELSE 0 END) AS den_p6m_curr,

        /* Year-ago windows */
        SUM(CASE WHEN f.day_date BETWEEN r.p1m_ya_start AND r.ya_end
                  AND f.in_flag = 'in' AND f.mfgr IN ('P&G','KAO','LION')
                 THEN f.pos_sales_amt ELSE 0 END) AS num_p1m_ya,
        SUM(CASE WHEN f.day_date BETWEEN r.p3m_ya_start AND r.ya_end
                  AND f.in_flag = 'in' AND f.mfgr IN ('P&G','KAO','LION')
                 THEN f.pos_sales_amt ELSE 0 END) AS num_p3m_ya,
        SUM(CASE WHEN f.day_date BETWEEN r.p6m_ya_start AND r.ya_end
                  AND f.in_flag = 'in' AND f.mfgr IN ('P&G','KAO','LION')
                 THEN f.pos_sales_amt ELSE 0 END) AS num_p6m_ya,

        SUM(CASE WHEN f.day_date BETWEEN r.p1m_ya_start AND r.ya_end
                  AND f.in_flag = 'in'
                 THEN f.pos_sales_amt ELSE 0 END) AS den_p1m_ya,
        SUM(CASE WHEN f.day_date BETWEEN r.p3m_ya_start AND r.ya_end
                  AND f.in_flag = 'in'
                 THEN f.pos_sales_amt ELSE 0 END) AS den_p3m_ya,
        SUM(CASE WHEN f.day_date BETWEEN r.p6m_ya_start AND r.ya_end
                  AND f.in_flag = 'in'
                 THEN f.pos_sales_amt ELSE 0 END) AS den_p6m_ya

    FROM fact f
    CROSS JOIN ranges r
    GROUP BY f.category
)

SELECT
    a.category AS `Category`,

    /* P1M (category share) */
    ROUND(a.num_p1m_curr / NULLIF(a.den_p1m_curr, 0), 4) AS `P1M`,

    /* P1M_DYA (category share delta vs year-ago) */
    ROUND(
        (a.num_p1m_curr / NULLIF(a.den_p1m_curr, 0))
      - (a.num_p1m_ya   / NULLIF(a.den_p1m_ya,   0)), 4
    ) AS `P1M_DYA`,

    /* vs National Share (delta): category P1M share - national P1M share */
    ROUND(n.num_nat_p1m_curr / NULLIF(n.den_nat_p1m_curr, 0), 4
    ) AS `vs National Share`,

    /* P3M + DYA */
    ROUND(a.num_p3m_curr / NULLIF(a.den_p3m_curr, 0), 4) AS `P3M`,
    ROUND(
        (a.num_p3m_curr / NULLIF(a.den_p3m_curr, 0))
      - (a.num_p3m_ya   / NULLIF(a.den_p3m_ya,   0)), 4
    ) AS `P3M_DYA`,

    /* P6M + DYA */
    ROUND(a.num_p6m_curr / NULLIF(a.den_p6m_curr, 0), 4) AS `P6M`,
    ROUND(
        (a.num_p6m_curr / NULLIF(a.den_p6m_curr, 0))
      - (a.num_p6m_ya   / NULLIF(a.den_p6m_ya,   0)), 4
    ) AS `P6M_DYA`

FROM agg a
JOIN nat n on a.category=n.category       -- scalar (same national share across rows), used only to compute delta
ORDER BY `Category`;